# Parsing Palm Detection Full model

In [ ]:
# General imports used throughout the tutorial

# file operations
import json
import os

import numpy as np

from IPython.display import SVG

from PIL import Image, ImageOps
from matplotlib import pyplot as plt

# import the ClientRunner class from the hailo_sdk_client package
from hailo_sdk_client import ClientRunner

### Set the hardware architecture to be used in this notebook

In [ ]:
chosen_hw_arch = "hailo8"
# For Hailo-15 devices, use 'hailo15h'
# For Mini PCIe modules or Hailo-8R devices, use 'hailo8r'

### model name definitions

In [ ]:
model_name = "palm_detection_full"
hailo_model_har_name = f"{model_name}_hailo_model.har"

### Generate calibration dataset

The calibration dataset should be preprocessed according yo the model's input requirements (in this case, 194x194), and it is recommended to have at least 1024 images and to use a GPU. In this tutorial, we use 1024 jpeg images under ```images``` directory, and convert it to npy file.

**Note:**

The imagery dataset used here is a subset of the [Human Images Dataset - Men and Women](https://www.kaggle.com/datasets/snmahsa/human-images-dataset-men-and-women), available on Kaggle under the [MIT license](https://www.mit.edu/~amini/LICENSE.md).

Please download the dataset, then extract and place the dataset under `notebook` directory as below. Images should be at least 1024 images.

```
notebook
├── gender_dataset
│   ├── men
│   │   ├── 1.jpg
│   │   ├── 2.jpg
│   │   ├── 3.jpg
│   │   ...
│   │   ├── 512.jpg
│   │   ├── 513.jpg
│   │   └── 514.jpg
│   └── women
│       ├── 1.jpg
│       ├── 2.jpg
│       ├── 3.jpg
│       ...
│       ├── 511.jpg
│       ├── 512.jpg
│       └── 513.jpg
├── models
│   └── palm_detection_full.tflite
├── palm_detection_full_DFC.ipynb
└── palm_detection_full_inference.ipynb
```

In [ ]:
def preproc(
    image: np.ndarray, output_height: int, output_width: int
) -> np.ndarray:
    """
    Resize an image to the specified dimensions using NumPy and Pillow.

    Parameters:
    - image (np.ndarray): The input image as a NumPy array with shape [height, width, channels].
    - output_height (int): Desired height of the output image.
    - output_width (int): Desired width of the output image.

    Returns:
    - np.ndarray: Processed image with dimensions [output_height, output_width, channels].
    """

    # Resize using Pillow with bilinear interpolation (default in PIL.Image.resize)
    pil_image = Image.fromarray(image)
    resized_image = pil_image.resize((output_width, output_height), resample=Image.Resampling.BILINEAR)

    return np.array(resized_image).astype(np.uint8)


def preproc_with_pad(
    image: np.ndarray, output_height: int, output_width: int
) -> np.ndarray:
    """
    Resize an image to the specified dimensions using NumPy and Pillow.
    Keep the aspect retio of the original image when resize and add pads to adjust to the output dimensions.

    Parameters:
    - image (np.ndarray): The input image as a NumPy array with shape [height, width, channels].
    - output_height (int): Desired height of the output image.
    - output_width (int): Desired width of the output image.

    Returns:
    - np.ndarray: Processed image with dimensions [output_height, output_width, channels].
    """
    h, w = image.shape[:2]

    # Calculate the new side length for resizing while preserving aspect ratio
    resize_side = max(h, w)

    # Determine scaling factor based on aspect ratio
    if h < w:
        scale = output_width / w
    else:
        scale = output_height / h

    # Resize using Pillow with bilinear interpolation (default in PIL.Image.resize)
    pil_image = Image.fromarray(image)
    new_h, new_w = int(h * scale), int(w * scale)
    resized_image = pil_image.resize((new_w, new_h), resample=Image.Resampling.BILINEAR)

    # Pad the image to target size with black color
    delta_w = output_width - new_w
    delta_h = output_height - new_h
    
    padding = (
        int(delta_w // 2),
        int(delta_h // 2),
        int(delta_w - (delta_w // 2)),
        int(delta_h - (delta_h // 2)),
        )
    cropped_image = ImageOps.expand(resized_image, padding)

    return np.array(cropped_image).astype(np.uint8)


def list_image_files(directory, extension):
    images_list = []
    
    # Walk through the directory tree
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(extension):
                full_path = os.path.join(root, file)
                images_list.append(full_path)

    return images_list
    

def process_images(directory, target_size, extension = (".png", ".jpg"), with_pad=True):
    """
    Process images in a specified directory and save them as a NumPy array.

    This function scans the given directory for image files with specified extensions,
    reads each image, ensures they are valid RGB images, and then processes them using
    the preproc function. It returns all processed images as a single 4D NumPy array.

    Parameters:
    - directory (str): The path to the directory containing images.
    - target_size (Tuple[int, int, float]): Desired size (height, width) for output images,
      along with the resize factor.
    - extension (Tuple[str], optional): Tuple of allowed image file extensions. Default is (".png", ".jpg").

    Returns:
    - np.ndarray: An array containing all processed images with shape
                  [num_images, target_height, target_width, channels].

    Raises:
    - FileNotFoundError: If the specified directory does not exist.
    - ValueError: If an image in the directory is not a valid RGB image.
    """
    if not os.path.exists(directory):
        raise FileNotFoundError(f"Directory {directory} does not exist.")


    images_list = list_image_files(directory, extension)

    if not images_list:
        raise ValueError("No valid images found in the directory.")

    num_images = len(images_list)
    processed_images = np.zeros((num_images, *target_size[:2], 3))

    for idx, img_name in enumerate(images_list):
        # Load and validate each image
        valid_image_mode = {"RGB", "RGBA", "P", "PA"}
        with Image.open(img_name) as img:
            if img.mode not in valid_image_mode:
                raise ValueError(f"Image \"{img_name}:{img.mode}\" is not a valid RGB image.")

            # If palettised image, convert to RGB image
            if img.mode == "RGBA" or img.mode == "P" or img.mode == "PA":
                print(f"Convert image \"{img_name}:{img.mode}\" to RGB image")
                img = img.convert('RGB')

            # Convert PIL image to NumPy array
            img_array = np.array(img)

            # Process the image using preproc function
            if with_pad:
                processed_images[idx] = preproc_with_pad(img_array, *target_size)
            else:
                processed_images[idx] = preproc(img_array, *target_size)

    return processed_images, images_list


calib_dataset = []
images_list = []

try:
    # Process images and save as NumPy array, palm detection full model input size is 192x192
    calib_dataset, images_list = process_images(
        "./gender_dataset",
        target_size=(192, 192),
        with_pad=False,
    )
    np.save("calib_dataset.npy", calib_dataset)
    num_of_calib_images = len(calib_dataset)
    print(f"Processed {num_of_calib_images} images saved to 'calib_dataset.npy'")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Check processed image from one of calibration images
img = np.array(Image.open(images_list[num_of_calib_images-1]))
plt.imshow(img, interpolation='nearest')
plt.title('Original image')
plt.show()

plt.imshow(np.array(calib_dataset[len(calib_dataset)-1,:,:,:], np.uint8), interpolation='nearest')
plt.title('Preprocessed image')
plt.show()

## Parsing Palm Detection Model from tflite to HAR

In [ ]:
tflite_path = f"./models/{model_name}.tflite"

Download tflite model from meidapipe to models directory

In [ ]:
!wget -O {tflite_path} https://storage.googleapis.com/mediapipe-assets/palm_detection_full.tflite

Parse the model with the default arguments

In [ ]:
runner = ClientRunner(hw_arch=chosen_hw_arch)
hn, npz = runner.translate_tf_model(
    tflite_path,
    model_name,
)

While parsing, we received the error ```"UnsupportedModelError: could not detect inputs to concatenate layer 'concat1' (translated from Identity1)."``` 

To prevent this parsing error, assign the ending node to the convolutional layer preceding the reshape-concatenate layer.

*Note:* You can use [Netron https://netron.app/](https://netron.app/) to check the model structure and node names.

In [ ]:
runner = ClientRunner(hw_arch=chosen_hw_arch)
hn, npz = runner.translate_tf_model(
    tflite_path,
    model_name,
    start_node_names=["input_1"],
    end_node_names=[
        "model_1/model/classifier_palm_16_NO_PRUNING/BiasAdd;model_1/model/classifier_palm_16_NO_PRUNING/Conv2D;model_1/model/classifier_palm_16_NO_PRUNING/BiasAdd/ReadVariableOp/resource1",
        "model_1/model/classifier_palm_8_NO_PRUNING/BiasAdd;model_1/model/classifier_palm_8_NO_PRUNING/Conv2D;model_1/model/classifier_palm_8_NO_PRUNING/BiasAdd/ReadVariableOp/resource1",
        "model_1/model/regressor_palm_16_NO_PRUNING/BiasAdd;model_1/model/regressor_palm_16_NO_PRUNING/Conv2D;model_1/model/regressor_palm_16_NO_PRUNING/BiasAdd/ReadVariableOp/resource1",
        "model_1/model/regressor_palm_8_NO_PRUNING/BiasAdd;model_1/model/regressor_palm_8_NO_PRUNING/Conv2D;model_1/model/regressor_palm_8_NO_PRUNING/BiasAdd/ReadVariableOp/resource1"],
    )

Save har model. (At this point, har contains only float32 model.)

In [ ]:
runner.save_har(hailo_model_har_name)

#### Visualize

In [ ]:
!hailo visualizer {hailo_model_har_name} --no-browser
SVG(f"{model_name}.svg")

## Optimize the model with 8bit quantization

In [ ]:
# First, we will load our parsed HAR
runner = ClientRunner(har=hailo_model_har_name)
# By default it uses the hw_arch that is saved on the HAR. For overriding, use the hw_arch flag.

**Note:** At first, you should optimize the model with ```optimization_level=0``` to check the model can be optimized without errors.

In [ ]:
# Now we will create a model script, that tells the compiler to add a normalization on the beginning
# of the model (that is why we didn't normalize the calibration set;
# Otherwise we would have to normalize it before using it)

#calibration dataset
calib_dataset = np.load('calib_dataset.npy')

# Batch size is 8 by default
alls_lines = [
    "normalization1 = normalization([0.0, 0.0, 0.0], [255.0, 255.0, 255.0])\n",
    "model_optimization_flavor(optimization_level=2, compression_level=0, batch_size=8)\n",
]

# Load the model script to ClientRunner so it will be considered on optimization
runner.load_model_script("".join(alls_lines))

# Call Optimize to perform the optimization process
runner.optimize(calib_dataset)

# Save the result state to a Quantized HAR file
quantized_model_har_path = f"{model_name}_quantized_model.har"
runner.save_har(quantized_model_har_path)

## Compile quantized Palm Detection Full model

In [ ]:
quantized_model_har_path = f"{model_name}_quantized_model.har"
runner = ClientRunner(har=quantized_model_har_path)
# By default it uses the hw_arch that is saved on the HAR. It is not recommended to change the hw_arch after Optimization.

hef = runner.compile()

hef_name = f"../hefs/{model_name}.hef"
with open(hef_name, "wb") as f:
    f.write(hef)

If you can compile the model without errors, you can test the compiled hef model with HailoRT in the next notebook!